In [ ]:
!pip install pyshp

In [ ]:
!pip install geopandas
!pip install descartes

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import geopandas as gpd
import seaborn as sns
sns.set(style='whitegrid', palette='pastel', color_codes=True)
sns.mpl.rc('figure', figsize=(10,6))

In [ ]:
# opening the vector map
shp_path = "../z_resources/data_test/taxi_zones/taxi_zones.shp"
assert os.path.exists(shp_path), "Input file does not exist."

# set the filepath and load
shapefile = "../z_resources/data_test/taxi_zones/taxi_zones.shp"
# reading the shape file, importing just 4 columns
map_df = gpd.read_file(shapefile)[['zone', 'LocationID', 'borough', 'geometry']]

# filter Manhattan zones
map_df = map_df[map_df['borough'] == 'Manhattan']

# check data type so we can see that this is not a normal dataframe, but a GEOdataframe
print(type(map_df))
map_df.head(70)

In [ ]:
# filter Manhattan zones
map_df2 = map_df[map_df['borough'] == 'Manhattan']

# remove duplicated zone 103 (Liberty Island)
map_df2.drop([102,103], inplace=True)

map_df2.head(70)

In [ ]:
map_df2.plot()

In [ ]:
data = pd.DataFrame({'RandVariable':np.random.randn(67)})

map_df2.reset_index(inplace=True, drop=True)
data.reset_index(inplace=True, drop=True)
print(map_df2.shape, data.shape)
display(map_df2.head(2), data.head(2))
map_df2 = map_df2.join(data)
map_df2.head()

In [ ]:
# set the range for the choropleth
vmin, vmax = data.min(), data.max()# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 15))

map_df2.plot(column='RandVariable', cmap='BuGn', linewidth=0.8, ax=ax, edgecolor='1')

# Create colorbar as a legend
colorBar = plt.cm.ScalarMappable(cmap='BuGn', norm=plt.Normalize(vmin=vmin, vmax=vmax))

# add the colorbar to the figure
cbar = fig.colorbar(colorBar)

### Incorporate Bokeh library to introduce interactivity
Bokeh consumes GeoJSON format so I´ll convert the GeoDataFrame to GeoJSON

In [ ]:
import json

#Read data to json.
map_df2_json = json.loads(map_df2.to_json())

# convert to String like  object
map_df2_json_data = json.dumps(map_df2_json)

#### Function to get x and y coordinates from the geometry polygon

In [ ]:
map_df2['geometry'][11][1].exterior.coords.xy[1]

In [ ]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""
    polygonList = []
    
    for row in row[geom]:
        if row.type == 'MultiPolygon':
            for polygone in row.geoms:
            polygoneList.append(polygone)
        else:
            polygoneList.append(row)
    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [ ]:
map_df2['x'] = map_df2.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
map_df2['y'] = map_df2.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)
map_df2.head()

### Render choropeth map using Bokeh

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
import json

#Read data to json.
map_df2_json = json.loads(map_df2.to_json())

# convert to String like  object
map_df2_json_data = json.dumps(map_df2_json)

# 1. Create GeoJSONDataSource
geosource = GeoJSONDataSource(geojson = map_df2_json_data)

# 2. define colour palette
# https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3
# [10] number of colour in the palette
palette = brewer['OrRd'][8]

# 2.1 reverse number of colours so the dark colour represents the higher value
palette = palette[::-1]

# 2.2. Instantiate LinearColorMapper that linearly maps numbers in a range into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = float(data.min()), high = float(data.max()))

# Create colour bar
color_bar = ColorBar(color_mapper = color_mapper,
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0),
                    orientation = 'horizontal')

# Create figure object
f = figure(title = 'Taxi Demand in Manhattan',
          plot_height = 600, plot_width = 950, toolbar_location = None)
f.xgrid.grid_line_color = None
f.ygrid.grid_line_color = None

# Add patch renderer to the figure
#fill_color = {'field':'RendVariable', 'transform':color_mapper}
f.patches('xs','ys', source = geosource,
          fill_color = None,
         line_color = 'black',
         line_width = '0.25')

# specify figure layout
f.add_layout(color_bar, 'below')

#output_notebook()
show(f)

In [ ]:
f.multi_polygons

# #########Walkthrough
I will follow this tutorial to learn mapping with bokeh and GeoPandas

https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

In [ ]:
!pip install geopandas
!pip install bokeh

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [36]:
# Read in shapefile and examine data
contiguous_usa = gpd.read_file('../z_resources/data_test/walkthrough/cb_2018_us_state_20m.shp')
# same with my NY shapefile filtering only Manhattan
contiguous_ny = gpd.read_file('../z_resources/data_test/taxi_zones/taxi_zones.shp')
contiguous_ny = contiguous_ny[contiguous_ny['borough'] == 'Manhattan'].reset_index(drop=True)

print(contiguous_usa.shape, contiguous_ny.shape)
display(contiguous_usa.head(), contiguous_ny.head())

(52, 10) (69, 7)


,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,24,01714934,0400000US24,24,MD,Maryland,00,25151100280,6979966958,"MULTIPOLYGON (((-76.04621 38.02553, -76.00734 ..."
1,19,01779785,0400000US19,19,IA,Iowa,00,144661267977,1084180812,"POLYGON ((-96.62187 42.77925, -96.57794 42.827..."
2,10,01779781,0400000US10,10,DE,Delaware,00,5045925646,1399985648,"POLYGON ((-75.77379 39.72220, -75.75323 39.757..."
3,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702,"MULTIPOLYGON (((-82.86334 41.69369, -82.82572 ..."
4,42,01779798,0400000US42,42,PA,Pennsylvania,00,115884442321,3394589990,"POLYGON ((-80.51989 40.90666, -80.51964 40.987..."


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
1,12,0.036661,0.000042,Battery Park,12,Manhattan,"POLYGON ((979908.772 196066.565, 979980.852 19..."
2,13,0.050281,0.000149,Battery Park City,13,Manhattan,"POLYGON ((980801.310 201248.869, 980697.386 20..."
3,24,0.047000,0.000061,Bloomingdale,24,Manhattan,"POLYGON ((995453.114 230274.267, 995312.583 23..."
4,41,0.052793,0.000143,Central Harlem,41,Manhattan,"POLYGON ((998716.913 234240.397, 999458.736 23..."


In [43]:
# Read in state population data and examine
state_pop = pd.read_csv('../z_resources/data_test/walkthrough/state_pop_2018.csv')

# create random data to merge with NY
ny_data = pd.DataFrame({'NY_data':np.random.uniform(low=0, high=10,size=contiguous_ny.shape[0])})

print(state_pop.shape, ny_data.shape)
display(state_pop.head(), ny_data.head())

(53, 8) (69, 1)


,SUMLEV,REGION,DIVISION,STATE,NAME,POPESTIMATE2018,POPEST18PLUS2018,PCNT_POPEST18PLUS
0,10,0,0,0,United States,327167434,253768092,77.6
1,40,3,6,1,Alabama,4887871,3798031,77.7
2,40,4,9,2,Alaska,737438,553622,75.1
3,40,4,8,4,Arizona,7171646,5528989,77.1
4,40,3,7,5,Arkansas,3013825,2310645,76.7


,NY_data
0,7.251447
1,9.309339
2,8.352560
3,6.577801
4,3.303946


In [44]:
# Merge shapefile with population data
pop_states = contiguous_usa.merge(state_pop, left_on = 'NAME', right_on = 'NAME')
# Drop Alaska and Hawaii
pop_states = pop_states.loc[~pop_states['NAME'].isin(['Alaska', 'Hawaii'])]

# Merge NY shapefile with ny_data
ny_zones = contiguous_ny.join(ny_data)

print(pop_states.shape, ny_zones.shape)
display(pop_states.head(), ny_zones.head(70))

(49, 17) (69, 8)


,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,SUMLEV,REGION,DIVISION,STATE,POPESTIMATE2018,POPEST18PLUS2018,PCNT_POPEST18PLUS
0,24,01714934,0400000US24,24,MD,Maryland,00,25151100280,6979966958,"MULTIPOLYGON (((-76.04621 38.02553, -76.00734 ...",40,3,5,24,6042718,4702570,77.8
1,19,01779785,0400000US19,19,IA,Iowa,00,144661267977,1084180812,"POLYGON ((-96.62187 42.77925, -96.57794 42.827...",40,2,4,19,3156145,2425378,76.8
2,10,01779781,0400000US10,10,DE,Delaware,00,5045925646,1399985648,"POLYGON ((-75.77379 39.72220, -75.75323 39.757...",40,3,5,10,967171,763555,78.9
3,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702,"MULTIPOLYGON (((-82.86334 41.69369, -82.82572 ...",40,2,3,39,11689442,9096117,77.8
4,42,01779798,0400000US42,42,PA,Pennsylvania,00,115884442321,3394589990,"POLYGON ((-80.51989 40.90666, -80.51964 40.987...",40,1,2,42,12807060,10158149,79.3


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,NY_data
0,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20...",7.251447
1,12,0.036661,0.000042,Battery Park,12,Manhattan,"POLYGON ((979908.772 196066.565, 979980.852 19...",9.309339
2,13,0.050281,0.000149,Battery Park City,13,Manhattan,"POLYGON ((980801.310 201248.869, 980697.386 20...",8.352560
3,24,0.047000,0.000061,Bloomingdale,24,Manhattan,"POLYGON ((995453.114 230274.267, 995312.583 23...",6.577801
4,41,0.052793,0.000143,Central Harlem,41,Manhattan,"POLYGON ((998716.913 234240.397, 999458.736 23...",3.303946
...,...,...,...,...,...,...,...,...
64,246,0.069467,0.000281,West Chelsea/Hudson Yards,246,Manhattan,"POLYGON ((983031.177 217138.506, 983640.320 21...",3.567522
65,249,0.036384,0.000072,West Village,249,Manhattan,"POLYGON ((983555.319 204876.901, 983469.158 20...",5.554519
66,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19...",3.225304
67,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824....",3.205783


In [45]:
# Transform geo dataframes to GeoJSONDataSource

In [46]:
import json
from bokeh.io import show, output_notebook
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

# Input GeoJSON source that contains features for plotting
geosource_states = GeoJSONDataSource(geojson = pop_states.to_json())

# the same with NY data
geosource_ny = GeoJSONDataSource(geojson = ny_zones.to_json())

In [47]:
# STATE POPULATION
# Create figure object.
output_notebook()
p = figure(title = 'Lead Levels in Water Samples, 2018', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None# Add patch renderer to figure.

states = p.patches('xs','ys', source = geosource_states,
                   fill_color = None,
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)# Create hover tool

p.add_tools(HoverTool(renderers = [states],
                     tooltips = [('State','@NAME'),
                            ('Population','@POPESTIMATE2018')]))
show(p)

Loading BokehJS ...

In [48]:
# NY DATA
# Create figure object.
output_notebook()
p_ny = figure(title = 'New York Taxis', 
           plot_height = 950 ,
           plot_width = 600, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')

p_ny.xgrid.grid_line_color = None
p_ny.ygrid.grid_line_color = None# Add patch renderer to figure.

p_ny_zones = p_ny.patches('xs','ys', source = geosource_ny,
                   fill_color = None,
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)# Create hover tool

p_ny.add_tools(HoverTool(renderers = [p_ny_zones],
                     tooltips = [('Zone','@LocationID'),
                            ('RandomData','@NY_data')]))
show(p_ny)

Loading BokehJS ...